In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
pd.pandas.set_option('display.max_columns',None)

In [ ]:
## Loading the dataset
DATA_PATH = '../input/house-prices-advanced-regression-techniques/'
df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

# Explorary Data Analysis

In [ ]:
df = pd.concat([df_test.assign(ind="test"), df_train.assign(ind="train")])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
nan_cols = [i for i in df.columns if df[i].isnull().sum()>=1]
print(nan_cols)

In [ ]:
pd.set_option('display.max_rows', None)
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values(['percent_missing'], ascending=False)

In [ ]:
df = df.loc[:, df.isnull().mean() < .9]
df.shape

In [ ]:
df_train_categorical = df.select_dtypes(include=['object']).columns.tolist() 
print(len(df_train_categorical))

In [ ]:
df_train_numeric = df.select_dtypes(exclude=['object']).columns.tolist()

In [ ]:
df_year_feature = [year_feature for year_feature in df_train_numeric if 'Yr' in year_feature or 'Year' in year_feature ]
print(df_year_feature)

In [ ]:
for feature in df_year_feature:
    if feature!= 'YrSold':
        data = df.copy()
        data[feature] = data['YrSold']-data[feature]
        plt.scatter(data[feature], data['SalePrice'])
        plt.xlabel(feature)
        plt.ylabel('SalePrice')
        plt.show()

In [ ]:
df_train.duplicated().sum()

In [ ]:
discrete_df = [feature for feature in df_train_numeric if len(df[feature].unique())<25 and feature not in df_year_feature+["ID"] ]

In [ ]:
for feature in discrete_df:
    N=25
    data=df.copy()
    data.groupby(feature)['SalePrice'].median().plot.bar(color=plt.cm.Paired(np.arange(N)))
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title(feature)
    plt.show()

In [ ]:
continous_df = [feature for feature in df_train_numeric if feature not in discrete_df+['Id']+df_year_feature+['SalePrice']]

In [ ]:
for feature in continous_df:
    data = df_train.copy()
    data.groupby(feature)['SalePrice'].median().plot.hist()
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title(feature)
    plt.show()

In [ ]:
for feature in df_train_categorical:
    N=25
    data=df.copy()
    data.groupby(feature)['SalePrice'].median().plot.bar(color=plt.cm.Paired(np.arange(N)))
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title(feature)
    plt.show()

# Feature Engineering
#### in this section we will perform below task
#### 1. Handle Missing values
#### 2. Handle Categorical Variable
#### 3. Handle Imbalance or skewd data
#### 4. Handle Outliers


In [ ]:
numerical_with_nan=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes!='O']
numerical_with_nan.remove('SalePrice')
numerical_with_nan

In [ ]:
for feature in numerical_with_nan:
    ## We will replace by using median since there are outliers
    median_value=df[feature].median()
    df[feature].fillna(median_value,inplace=True)
    
df[numerical_with_nan].isnull().sum()

# 2. Handling Tamporal Variable

In [ ]:
for feature in ['YearBuilt','YearRemodAdd','GarageYrBlt']:
       
    df[feature]=df['YrSold']-df[feature]

In [ ]:
df[['YearBuilt','YearRemodAdd','GarageYrBlt']].head()

In [ ]:
for feature in continous_df:
    data=df.copy()
    if 0 in data[feature].unique():
        pass
    else:
        data[feature]=np.log(data[feature])
        data.boxplot(column=feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.show()

In [ ]:
import numpy as np
num_features=['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea']

for feature in num_features:
    df[feature]=np.log(df[feature])

In [ ]:
for feature in num_features:
    sns.kdeplot(data=df_train, x=feature,  legend=True)
    plt.show()

In [ ]:
num_feature=['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']

def iqr_feature(features, df):
    dict = {}
    max_lmt = []
    min_lmt=[]
    for feature in features:
        q1 = df[feature].quantile(0.25)
        q2 = df[feature].quantile(0.75)
        IQR = q2 - q1
#         iqr.append(IQR)
        max_limit = q2 + (1.5 * IQR)
        max_lmt.append(max_limit)
        min_limit = q1 - (1.5 * IQR) 
        min_lmt.append(min_limit)
#         dict.update({feature: [min_limit,max_limit]})
#     return dict
    return max_lmt, min_lmt
    
max_lmt, min_lmt= iqr_feature(num_feature, df)
min_lmt

In [ ]:
s1 = pd.Series(max_lmt)
s2 = pd.Series(min_lmt)


df_min_max = pd.DataFrame(list(zip(num_feature,s1, s2)), columns=['num_features','max_lmt','min_lmt'])
df_min_max
    

In [ ]:
df.shape

In [ ]:
# df_new = df[(df["LotFrontage"]>3.700798) & (df["LotFrontage"]<4.750255)]
# df_new.shape
df["LotFrontage"]= np.where((df["LotFrontage"]<3.700798), df['LotFrontage'].quantile(0.05),df['LotFrontage'])
df["LotFrontage"]= np.where((df["LotFrontage"]>4.750255), df['LotFrontage'].quantile(0.95),df['LotFrontage'])
df.shape

In [ ]:
sns.boxplot( y=df["LotFrontage"]);
plt.show()

In [ ]:
df["LotArea"]= np.where((df["LotArea"]<8.65046), df['LotArea'].quantile(0.05),df['LotArea'])
df["LotArea"]= np.where((df["LotArea"]>10.010846), df['LotArea'].quantile(0.95),df['LotArea'])
sns.boxplot( y=df["LotArea"]);
plt.show()

In [ ]:
df["1stFlrSF"]= np.where((df["1stFlrSF"]<6.085527), df['1stFlrSF'].quantile(0.05),df['1stFlrSF'])
df["1stFlrSF"]= np.where((df["1stFlrSF"]>7.925098), df['1stFlrSF'].quantile(0.95),df['1stFlrSF'])
df.shape

In [ ]:
df["GrLivArea"]= np.where((df["GrLivArea"]<6.370592), df['GrLivArea'].quantile(0.05),df['GrLivArea'])
df["GrLivArea"]= np.where((df["GrLivArea"]>8.119484), df['GrLivArea'].quantile(0.95),df['GrLivArea'])
df.shape

In [ ]:
df["SalePrice"]= np.where((df["SalePrice"]<3937.500000), df['SalePrice'].quantile(0.05),df['SalePrice'])
df["SalePrice"]= np.where((df["SalePrice"]>340037.500000), df['SalePrice'].quantile(0.95),df['SalePrice'])
df.shape

In [ ]:
df['SalePrice'].tail()

# Handling Rare Categorical Feature

We will remove categorical variables that are present less than 1% of the observations

In [ ]:
categorical_features=[feature for feature in df.columns if df[feature].dtype=='O']
categorical_features.remove('ind')

In [ ]:
for feature in categorical_features:
    print((df[feature].value_counts()))

In [ ]:
for feature in categorical_features:
    temp=df.groupby(feature)['SalePrice'].count()/len(df)
    temp_df=temp[temp>0.01].index
    df[feature]=np.where(df[feature].isin(temp_df),df[feature],'Rare_var')
    

In [ ]:
# from sklearn.preprocessing import OrdinalEncoder

# Target Guided Ordinal Encoding
1. Ordering the labels according to the target
2. Replace the labels by the joint probability of being 1 or 0

In [ ]:
for feature in categorical_features:
    labels_ordered=df.groupby([feature])['SalePrice'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    df[feature]=df[feature].map(labels_ordered)

In [ ]:
df.head()

In [ ]:
test, train = df[df["ind"].eq("test")], df[df["ind"].eq("train")]

In [ ]:
test.drop('SalePrice', axis=1, inplace=True)

In [ ]:
train.drop('ind', axis=1, inplace=True)

In [ ]:
nan_feature = [feature for feature in test.columns if test[feature].isnull().sum()>=1]
nan_feature.remove('GarageCars')

In [ ]:
for feature in nan_feature:
    ## We will replace by using median since there are outliers
    median_value=df[feature].median()
    test[feature].fillna(median_value,inplace=True)
    
test[nan_feature].isnull().sum()

In [ ]:
md = test['GarageCars'].mode()
print(md)

In [ ]:
test['GarageCars'].fillna(2,inplace=True)

In [ ]:
test.isnull().sum()

In [ ]:
test.drop('ind', axis=1, inplace=True)

In [ ]:
y_data=train['SalePrice']
x_data=train.drop('SalePrice',axis=1)

In [ ]:
x_data.shape, test.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)
x_train.isnull().sum()

# Feature Selection

In [ ]:
from sklearn.feature_selection import SelectPercentile

In [ ]:
from sklearn.feature_selection import mutual_info_regression
# determine the mutual information
mutual_info = mutual_info_regression(x_train, y_train)
mutual_info

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = x_train.columns
mutual_info.sort_values(ascending=False)

In [ ]:
## Selecting the top 80 percentile
selected_top_columns = SelectPercentile(mutual_info_regression, percentile=80)
selected_top_columns.fit(x_train, y_train)

In [ ]:
X_train_top_80 = list(x_train.columns[selected_top_columns.get_support()])
len(X_train_top_80)

In [ ]:
X_train_feature = x_train[x_train.columns[x_train.columns.isin(X_train_top_80)]]
X_test_feature = x_test[x_test.columns[x_test.columns.isin(X_train_top_80)]]
df_test_feature = test[test.columns[test.columns.isin(X_train_top_80)]]

In [ ]:
# df_test_scaled = df_test_scaled.drop(df_test_scaled.index[-1])

# Model Training

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_feature , y_train)
predictions = model.predict(X_test_feature)

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score, r2_score

In [ ]:
print('Mean Absolute Error(MAE):', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error(MSE):', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('Explained Variance Score (EVS):',explained_variance_score(y_test,predictions))
print('R2:',metrics.r2_score(y_test, predictions))

In [ ]:
from sklearn.linear_model import Ridge

model_ridge = Ridge()
model_ridge.fit(X_train_feature , y_train)
ridge_predictions = model_ridge.predict(X_test_feature)

In [ ]:
print('Mean Absolute Error(MAE):', metrics.mean_absolute_error(y_test, ridge_predictions))
print('Mean Squared Error(MSE):', metrics.mean_squared_error(y_test, ridge_predictions))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, ridge_predictions)))
print('Explained Variance Score (EVS):',explained_variance_score(y_test,ridge_predictions))
print('R2:',metrics.r2_score(y_test, ridge_predictions))
print('R2 rounded:',(metrics.r2_score(y_test, ridge_predictions)).round(2))
r2 = r2_score(y_test, ridge_predictions)
r2_rounded = r2_score(y_test, ridge_predictions).round(2)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model_DecisionTree = DecisionTreeRegressor()
model_DecisionTree.fit(X_train_feature , y_train)
DecisionTree_predictions = model_DecisionTree.predict(X_test_feature)

In [ ]:
print('Mean Absolute Error(MAE):', metrics.mean_absolute_error(y_test, DecisionTree_predictions))
print('Mean Squared Error(MSE):', metrics.mean_squared_error(y_test, DecisionTree_predictions))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, DecisionTree_predictions)))
print('Explained Variance Score (EVS):',explained_variance_score(y_test,DecisionTree_predictions))
print('R2:',metrics.r2_score(y_test, DecisionTree_predictions))
print('R2 rounded:',(metrics.r2_score(y_test, DecisionTree_predictions)).round(2))
r2 = r2_score(y_test, DecisionTree_predictions)
r2_rounded = r2_score(y_test, DecisionTree_predictions).round(2)

In [ ]:
n_estimators = [5,20,50,100] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap': bootstrap}

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
               n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)

In [ ]:
rf_random.fit(X_train_feature , y_train)
RandomForest_predictions = rf_random.predict(X_test_feature)

In [ ]:
print('Mean Absolute Error(MAE):', metrics.mean_absolute_error(y_test, RandomForest_predictions))
print('Mean Squared Error(MSE):', metrics.mean_squared_error(y_test, RandomForest_predictions))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, RandomForest_predictions)))
print('Explained Variance Score (EVS):',explained_variance_score(y_test,RandomForest_predictions))
print('R2:',metrics.r2_score(y_test, RandomForest_predictions))
print('R2 rounded:',(metrics.r2_score(y_test, RandomForest_predictions)).round(2))
r2 = r2_score(y_test, RandomForest_predictions)
r2_rounded = r2_score(y_test, RandomForest_predictions).round(2)

In [ ]:
predictions = rf_random.predict(df_test_feature)

In [ ]:
submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predictions})

In [ ]:
submission.to_csv(path_or_buf = 'submission.csv', \
                   sep = ',', index = False)

In [ ]:
submission